<a href="https://colab.research.google.com/github/Tclack88/Renewable-Energy/blob/master/geothermal_webscraping_for_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
url = "https://rredc.nrel.gov/solar/old_data/nsrdb/1961-1990/redbook/sum2/23129.txt"  # major city solar (and wind speed)
# https://nsrdb.nrel.gov/api-instructions   # THIS API may be useful
# https://data.nrel.gov/submissions/54   # wind data
# https://gdr.openei.org/submissions/194    # geotherm ALL here
!wget "https://rredc.nrel.gov/solar/old_data/nsrdb/1961-1990/redbook/sum2/23129.txt"

--2019-10-09 21:18:45--  https://rredc.nrel.gov/solar/old_data/nsrdb/1961-1990/redbook/sum2/23129.txt
Resolving rredc.nrel.gov (rredc.nrel.gov)... 192.174.63.81, 2620:d0:8000:7::81
Connecting to rredc.nrel.gov (rredc.nrel.gov)|192.174.63.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5702 (5.6K) [text/plain]
Saving to: ‘23129.txt’

23129.txt           100%[===================>]   5.57K  --.-KB/s    in 0s      

2019-10-09 21:18:51 (332 MB/s) - ‘23129.txt’ saved [5702/5702]



In [0]:
dat = '23129.txt'
!tail -n 10 23129.txt > test.txt

In [0]:
import pandas as pd

dat_test = pd.read_csv("test.txt")
dat_test.head()

,Element,,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
0,Temp.,(deg C),13.3,14.1,14.7,16.5,18.2,20.3,22.8,23.6,22.4,20.1,16.3,13.3,17.9
1,Daily Min,(deg C),7.2,8.3,9.4,11.0,13.5,15.4,17.4,18.2,17.1,14.3,10.2,7.2,12.4
2,Daily Max,(deg C),19.3,19.8,20.0,21.9,22.9,25.0,28.2,28.9,27.8,25.8,22.3,19.4,23.4
3,Record Lo,(deg C),-3.9,0.6,0.6,3.3,4.4,8.3,10.6,11.1,10.0,3.9,1.1,-2.2,-3.9
4,Record Hi,(deg C),32.8,32.8,36.7,40.6,39.4,42.8,41.7,40.6,43.3,43.9,38.3,33.3,43.9


In [0]:

geo_instal = pd.read_excel('https://gdr.openei.org/files/194/GEOTHERM_ALL.xls')
geo_instal.head()
geo_instal = geo_instal[['STATE','LATITUDE','LONGITUDE','TEMP']]

In [0]:
geo_instal = geo_instal.dropna()

In [0]:
import plotly.express as px
import matplotlib.pyplot as plt

In [0]:
geo_instal.dtypes

STATE         object
LATITUDE      object
LONGITUDE     object
TEMP         float64
dtype: object

In [0]:
def convert(x):
  x = x.strip()
  allowed = '01234556789.'
  if x[-1] == 'W':
    factor = -1
  else:
    factor = 1
  x = ''.join(c for c in list(str(x).replace('-','.')) if c in allowed) # remove all chars but '.' and digits
  x = x.split('.')
  x = float('.'.join(x[0:2]))
  return factor*x

geo_instal.LATITUDE = geo_instal.LATITUDE.apply(convert)
geo_instal.LONGITUDE = geo_instal.LONGITUDE.apply(convert)

# Remove Outliers
geo_instal = geo_instal[(geo_instal.LATITUDE != 57.13) & (geo_instal.LATITUDE != -45.28)]
geo_instal = geo_instal[geo_instal.LONGITUDE < 104]
geo_instal = geo_instal[(geo_instal.TEMP < 3000) & (geo_instal.TEMP > 50)]  # two temperatures were at about 3000 and 25000. Temperatures below 50 aren't super useful

In [0]:
fig = plt.figure(figsize=(10,30))
fig = px.scatter_mapbox(data_frame=geo_instal, lat='LATITUDE',lon='LONGITUDE',color='TEMP')
fig.update_layout(mapbox_style='open-street-map')

<Figure size 720x2160 with 0 Axes>

In [0]:
geo_instal.head()

,STATE,LATITUDE,LONGITUDE,TEMP
2,ALASKA,59.44,-153.13,92.2
3,ALASKA,51.58,-176.37,63.0
4,ALASKA,51.58,-176.37,71.0
5,ALASKA,54.08,-165.38,42.8
6,ALASKA,54.09,-165.55,84.0


In [0]:
geo_instal.to_csv('renewable_energy_data.csv',index=False)